### SQLite Database Initialization

This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

### File and Configuration Setup

Additionally, it creates other essential files and configurations needed for subsequent operations.

In [ ]:
import json
import sqlite3

db_path = './auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()



### MySQL Items Data Retrieval

In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

### SQLite Database Update

After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.


In [ ]:
import json
import pandas as pd
import mysql.connector

with open('./config.json') as f:
    config = json.load(f)

mysql_db = mysql.connector.connect(**config['database'])

query = "SELECT * FROM Items"
df = pd.read_sql_query(query, mysql_db)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("DELETE FROM Items")
conn.commit()


### Auction Data and Items Storage

This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

### Data Import from Pandas DataFrame

Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [ ]:
import json
import os
from datetime import datetime
from tqdm import tqdm

data_dir = './data/migrator'
db = sqlite3.connect(db_path)
cursor = db.cursor()

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        if filename.endswith(".json") and filename != 'config.json':
            filepath = os.path.join(root, filename)
            try:
                data = json.load(open(filepath, "r"))
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"Error reading file {filepath}: {e}")
                continue

            print(f"Processing file: {filepath}")

            auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

            auctions_data = [
                (auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"])
                for auction in data["auctions"]
            ]
            action_events_data = [(auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')) for auction in data["auctions"]]

            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, auctions_data)
                db.commit()
                print(f"Auction data from file {filepath} successfully inserted into Auctions.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO ActionEvents (auction_id, record)
                    VALUES (?, ?)
                """, action_events_data)
                db.commit()
                print(f"Auction events for file {filepath} successfully inserted in ActionEvents.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

cursor.close()
db.close()
